In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

# 每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)  # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)   #标准差
        tf.summary.scalar('max', tf.reduce_max(var)) #最大值
        tf.summary.scalar('min', tf.reduce_min(var))  #最小值
        tf.summary.histogram('histogram', var)  # 直方图

#初始化权值
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)   #生成一个截断的正态分布
    return tf.Variable(initial, name=name)

#初始化偏置
def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial,name=name)

# 卷积层
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]  窗口
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 命名空间
with tf.name_scope('input'):
    # 定义两个placeholder
    x = tf.placeholder("float", [None, 784],name='x-input')  #28*28
    y = tf.placeholder("float", [None, 10],name='y-input')
    with tf.name_scope('x_image'):
        #改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]
        x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image')

with tf.name_scope('Conv1'):
    #初始化第一个卷积层的权值和偏置
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5,1,32], name='W_conv1')  #5*5的采样窗口，32个卷积核从 1 个平面抽取特征。  1个平面即 同x_image中1，因为黑白
    with tf.name_scope('b_conv1'):
        b_conv1= bias_variable([32], name='b_conv1')    #每一个卷积核一个偏置值

    #把x_image 和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image, W_conv1) + b_conv1
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)   # 进行max-pooling

with tf.name_scope('Conv2'):
    #初始化第二个卷积层的权值和偏置
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64], name='W_conv2')  # 64个卷积核从32个平面抽取特征
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64], name='b_conv2')
    
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(h_pool1, W_conv2) + b_conv2
    with tf.name_scope('relu'):
        #把h_pool1 和权值向量进行卷积
        h_conv2 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(h_conv2) 

#28*28 的图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14，第二次池化后变为7*7
#经过上面操作后得到64 张 7*7的特征平面

with tf.name_scope('fe1'):
    #初始化第一个全连接层的权值
    with tf.name_scope('W_fe1'):
        W_fe1 = weight_variable([7*7*64, 1024], name='W_fe1')  #上一层有7*7*64个神经元，全连接层有1024个神经元
    with tf.name_scope('b_fe1'):
        b_fe1 = bias_variable([1024], name='b_fe1')  #1024个节点

    with tf.name_scope('h_pool2_flat'):
        #把池化层2的输出扁平化为1维
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name='h_pool2_flat')
    with tf.name_scope('wx_plus_b1'):
        #求第一个全连接层的输出
        wx_plus_b1 = tf.matmul(h_pool2_flat, W_fe1) + b_fe1
    with tf.name_scope('relu'):
        h_fe1 = tf.nn.relu(wx_plus_b1)

    # keep_prob 用来表示神经元的输出概率
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder("float", name='keep_prob')
    with tf.name_scope('h_fe1_drop'):
        h_fe1_drop = tf.nn.dropout(h_fe1, keep_prob, name='h_fe1_drop')

with tf.name_scope('fe2'):
    #初始化第二个全连接层
    with tf.name_scope('W_fe2'):
        W_fe2 = weight_variable([1024, 10], name='W_fe2')
    with tf.name_scope('b_fe2'):
        b_fe2 = bias_variable([10], name='b_fe2')
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2 = tf.matmul(h_fe1_drop, W_fe2) + b_fe2
    with tf.name_scope('softmax'):
        #计算输出
        prediction = tf.nn.softmax(wx_plus_b2)

 # 使用交叉熵的方法改进 #########################################################
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction), name='cross_entropy')       
    tf.summary.scalar('cross_entropy', cross_entropy)
# 使用AdamOptimizer 进行优化
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)


# 求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))      # argmax 返回一维变量中最大值所在位置
    with tf.name_scope('accuracy'):
        # 求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tf.summary.scalar('accuracy', accuracy)

# 合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logs/train',sess.graph)
    test_writer = tf.summary.FileWriter('logs/test',sess.graph)
    for i in range(1001):
        #训练模型
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict = {x:batch_xs, y:batch_ys, keep_prob:0.5})
        #记录训练集计算的参数
        summary = sess.run(merged, feed_dict = {x:batch_xs, y:batch_ys, keep_prob:1.0})
        train_writer.add_summary(summary, i)
        #记录测试集计算的参数
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)
        summary = sess.run(merged, feed_dict = {x:batch_xs, y:batch_ys, keep_prob:1.0})
        test_writer.add_summary(summary, i)
        
        if i%100 == 0:
            test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
            train_acc = sess.run(accuracy, feed_dict={x: mnist.train.images[:10000], y: mnist.train.labels[:10000], keep_prob: 1.0})
            print("Iter" + str(i) + ", Test Accuracy = " + str(test_acc) + ", Train Accuracy = " + str(train_acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0, Test Accuracy = 0.1283, Train Accuracy = 0.1284
Iter100, Test Accuracy = 0.5597, Train Accuracy = 0.5499
Iter200, Test Accuracy = 0.7775, Train Accuracy = 0.771
Iter300, Test Accuracy = 0.9002, Train Accuracy = 0.8885
Iter400, Test Accuracy = 0.9308, Train Accuracy = 0.9197
Iter500, Test Accuracy = 0.944, Train Accuracy = 0.9348
Iter600, Test Accuracy = 0.9542, Train Accuracy = 0.9517
Iter700, Test Accuracy = 0.9554, Train Accuracy = 0.9562
Iter800, Test Accuracy = 0.9624, Train Accuracy = 0.959
Iter900, Test Accuracy = 0.9632, Train Accuracy = 0.961
Iter1000, Test Accuracy = 0.9681, Train Accuracy = 0.9651
